In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import plotly as plt
import plotly.offline as pltof
from plotly import tools
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Data source**:  The datasource in this project comes from THE WORLD BANK: https://data.worldbank.org/?name_desc=false     
  The analyse of this project is base on datasource above

US, China, Russia, Germany, UK, France, Italy, Saudi Arabia, S. Korea, Israel

# **Step 1: Data cleaning**







*1.1 Download and read the file*










In [2]:
Mili = pd.read_csv("Mili.csv",skiprows = [0,1,2,3],dtype={'Country Name':str,'Country Code': str}) # as first 4 line is something else that not related to the form
GDP = pd.read_csv("GDP.csv",skiprows = [0,1,2,3],dtype={'Country Name':str,'Country Code': str })
Pop = pd.read_csv("Populaton.csv",skiprows = [0,1,2,3],dtype={'Country Name':str,'Country Code': str})




*1.2 Selecting the Target colums*




In [3]:
col = ['Country Name','Country Code','2011','2012','2013','2014','2015','2016','2017','2018']
Mili = Mili[col]
GDP = GDP[col]
Pop = Pop[col]



*1.3 Select the target countries, year and creat the new dataset*


In [4]:
#Selecting the Target Countries
Countries = ['USA','CHN','RUS','DEU','GBR','FRA','ITA','SAU','KOR','ISR']
Mili = Mili[Mili['Country Code'].isin(Countries)] #Select dataset to correspond country
GDP = GDP[GDP['Country Code'].isin(Countries)]
Pop = Pop[Pop['Country Code'].isin(Countries)]

*1.4 Add column label before combine the dataset*

In [5]:
GDP.columns.values[2:] = ["GDP_" + str(col) for col in GDP.columns.values[2:]]
Pop.columns.values[2:] = ["Pop_" + str(col) for col in Pop.columns.values[2:]]
Mili.columns.values[2:] = ["Mili_" + str(col)for col in Mili.columns.values[2:]]

*1.5 Combine those dataset*


In [6]:

Pop = Pop.loc[:, Pop.columns != 'Pop_2011']       #drop  this value to merge the new dataset
GDP = GDP.loc[:, GDP.columns != 'GDP_2011']
Mili = Mili.loc[:, Mili.columns != 'Mili_2011']

data = pd.merge(GDP,Mili,on=['Country Name','Country Code'])
data = pd.merge(data,Pop,on=['Country Name','Country Code'],how = 'inner')
data = data.rename(columns={'Country Name':'Country_Name','Country Code':'Country_Code'})
data

,Country_Name,Country_Code,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017,GDP_2018,Mili_2012,Mili_2013,Mili_2014,Mili_2015,Mili_2016,Mili_2017,Mili_2018,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2018
0,China,CHN,8.532231e+12,9.570406e+12,1.047568e+13,1.106155e+13,1.123328e+13,1.231041e+13,1.389482e+13,1.573904e+11,1.798805e+11,2.007722e+11,2.140931e+11,2.160313e+11,2.278294e+11,2.499969e+11,1.350695e+09,1.357380e+09,1.364270e+09,1.371220e+09,1.378665e+09,1.386395e+09,1.392730e+09
1,Germany,DEU,3.527345e+12,3.732743e+12,3.883920e+12,3.360550e+12,3.466790e+12,3.665804e+12,3.949549e+12,4.647087e+10,4.593054e+10,4.610267e+10,3.981258e+10,4.157949e+10,4.538172e+10,4.947063e+10,8.042582e+07,8.064560e+07,8.098250e+07,8.168661e+07,8.234867e+07,8.265700e+07,8.290578e+07
2,France,FRA,2.683825e+12,2.811078e+12,2.852166e+12,2.438208e+12,2.471286e+12,2.595151e+12,2.787864e+12,6.003515e+10,6.241710e+10,6.361357e+10,5.534213e+10,5.735841e+10,6.041750e+10,6.379968e+10,6.565981e+07,6.599869e+07,6.631207e+07,6.654827e+07,6.672410e+07,6.686438e+07,6.696591e+07
3,United Kingdom,GBR,2.704888e+12,2.786023e+12,3.063803e+12,2.928591e+12,2.694283e+12,2.666229e+12,2.860668e+12,5.849566e+10,5.686176e+10,5.918286e+10,5.386219e+10,4.811894e+10,4.643330e+10,4.999719e+10,6.370022e+07,6.412827e+07,6.460230e+07,6.511622e+07,6.561159e+07,6.605886e+07,6.646034e+07
4,Israel,ISR,2.571801e+11,2.926362e+11,3.095585e+11,2.998131e+11,3.189506e+11,3.532534e+11,3.705880e+11,1.556710e+10,1.731971e+10,1.848583e+10,1.696943e+10,1.478381e+10,1.558161e+10,1.594679e+10,7.910500e+06,8.059500e+06,8.215700e+06,8.380100e+06,8.546000e+06,8.713300e+06,8.882800e+06
5,Italy,ITA,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12,1.875797e+12,1.961796e+12,2.085764e+12,2.978101e+10,2.995745e+10,2.770103e+10,2.218085e+10,2.503303e+10,2.644789e+10,2.780751e+10,5.953972e+07,6.023395e+07,6.078914e+07,6.073058e+07,6.062750e+07,6.053671e+07,6.042176e+07
6,"Korea, Rep.",KOR,1.278428e+12,1.370795e+12,1.484318e+12,1.465773e+12,1.500112e+12,1.623901e+12,1.720579e+12,3.195176e+10,3.431122e+10,3.755233e+10,3.657077e+10,3.688528e+10,3.917068e+10,4.306997e+10,5.019985e+07,5.042889e+07,5.074666e+07,5.101495e+07,5.121780e+07,5.136191e+07,5.160663e+07
7,Russian Federation,RUS,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12,1.276787e+12,1.574199e+12,1.669583e+12,8.146940e+10,8.835290e+10,8.469650e+10,6.641871e+10,6.924531e+10,6.652730e+10,6.138755e+10,1.432017e+08,1.435070e+08,1.438197e+08,1.440969e+08,1.443424e+08,1.444967e+08,1.444779e+08
8,Saudi Arabia,SAU,7.359748e+11,7.466471e+11,7.563503e+11,6.542699e+11,6.449355e+11,6.885861e+11,7.865218e+11,5.649787e+10,6.702000e+10,8.076240e+10,8.718587e+10,6.367280e+10,7.040000e+10,6.755467e+10,2.915519e+07,3.005252e+07,3.091699e+07,3.171767e+07,3.244257e+07,3.309915e+07,3.369995e+07
9,United States,USA,1.619701e+13,1.678485e+13,1.752175e+13,1.821930e+13,1.870719e+13,1.948539e+13,2.052905e+13,6.847800e+11,6.397040e+11,6.099140e+11,5.961046e+11,6.001064e+11,6.058029e+11,6.487983e+11,3.138310e+08,3.159937e+08,3.183010e+08,3.206352e+08,3.229413e+08,3.249855e+08,3.266875e+08


*1.6 Calculate the overal and percentage Military spending*

In [7]:
M_sum = data[['Mili_2012','Mili_2013','Mili_2014','Mili_2015','Mili_2016','Mili_2017','Mili_2018']]
data['Overall_Miliraty_spending'] = M_sum.sum(axis = 1)
data['Military_Percentage'] = data['Overall_Miliraty_spending'] / data['Overall_Miliraty_spending'].sum()
data


,Country_Name,Country_Code,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016,GDP_2017,GDP_2018,Mili_2012,Mili_2013,Mili_2014,Mili_2015,Mili_2016,Mili_2017,Mili_2018,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2018,Overall_Miliraty_spending,Military_Percentage
0,China,CHN,8.532231e+12,9.570406e+12,1.047568e+13,1.106155e+13,1.123328e+13,1.231041e+13,1.389482e+13,1.573904e+11,1.798805e+11,2.007722e+11,2.140931e+11,2.160313e+11,2.278294e+11,2.499969e+11,1.350695e+09,1.357380e+09,1.364270e+09,1.371220e+09,1.378665e+09,1.386395e+09,1.392730e+09,1.445994e+12,0.169794
1,Germany,DEU,3.527345e+12,3.732743e+12,3.883920e+12,3.360550e+12,3.466790e+12,3.665804e+12,3.949549e+12,4.647087e+10,4.593054e+10,4.610267e+10,3.981258e+10,4.157949e+10,4.538172e+10,4.947063e+10,8.042582e+07,8.064560e+07,8.098250e+07,8.168661e+07,8.234867e+07,8.265700e+07,8.290578e+07,3.147485e+11,0.036959
2,France,FRA,2.683825e+12,2.811078e+12,2.852166e+12,2.438208e+12,2.471286e+12,2.595151e+12,2.787864e+12,6.003515e+10,6.241710e+10,6.361357e+10,5.534213e+10,5.735841e+10,6.041750e+10,6.379968e+10,6.565981e+07,6.599869e+07,6.631207e+07,6.654827e+07,6.672410e+07,6.686438e+07,6.696591e+07,4.229835e+11,0.049668
3,United Kingdom,GBR,2.704888e+12,2.786023e+12,3.063803e+12,2.928591e+12,2.694283e+12,2.666229e+12,2.860668e+12,5.849566e+10,5.686176e+10,5.918286e+10,5.386219e+10,4.811894e+10,4.643330e+10,4.999719e+10,6.370022e+07,6.412827e+07,6.460230e+07,6.511622e+07,6.561159e+07,6.605886e+07,6.646034e+07,3.729519e+11,0.043793
4,Israel,ISR,2.571801e+11,2.926362e+11,3.095585e+11,2.998131e+11,3.189506e+11,3.532534e+11,3.705880e+11,1.556710e+10,1.731971e+10,1.848583e+10,1.696943e+10,1.478381e+10,1.558161e+10,1.594679e+10,7.910500e+06,8.059500e+06,8.215700e+06,8.380100e+06,8.546000e+06,8.713300e+06,8.882800e+06,1.146543e+11,0.013463
5,Italy,ITA,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12,1.875797e+12,1.961796e+12,2.085764e+12,2.978101e+10,2.995745e+10,2.770103e+10,2.218085e+10,2.503303e+10,2.644789e+10,2.780751e+10,5.953972e+07,6.023395e+07,6.078914e+07,6.073058e+07,6.062750e+07,6.053671e+07,6.042176e+07,1.889088e+11,0.022182
6,"Korea, Rep.",KOR,1.278428e+12,1.370795e+12,1.484318e+12,1.465773e+12,1.500112e+12,1.623901e+12,1.720579e+12,3.195176e+10,3.431122e+10,3.755233e+10,3.657077e+10,3.688528e+10,3.917068e+10,4.306997e+10,5.019985e+07,5.042889e+07,5.074666e+07,5.101495e+07,5.121780e+07,5.136191e+07,5.160663e+07,2.595120e+11,0.030473
7,Russian Federation,RUS,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12,1.276787e+12,1.574199e+12,1.669583e+12,8.146940e+10,8.835290e+10,8.469650e+10,6.641871e+10,6.924531e+10,6.652730e+10,6.138755e+10,1.432017e+08,1.435070e+08,1.438197e+08,1.440969e+08,1.443424e+08,1.444967e+08,1.444779e+08,5.180977e+11,0.060837
8,Saudi Arabia,SAU,7.359748e+11,7.466471e+11,7.563503e+11,6.542699e+11,6.449355e+11,6.885861e+11,7.865218e+11,5.649787e+10,6.702000e+10,8.076240e+10,8.718587e+10,6.367280e+10,7.040000e+10,6.755467e+10,2.915519e+07,3.005252e+07,3.091699e+07,3.171767e+07,3.244257e+07,3.309915e+07,3.369995e+07,4.930936e+11,0.057901
9,United States,USA,1.619701e+13,1.678485e+13,1.752175e+13,1.821930e+13,1.870719e+13,1.948539e+13,2.052905e+13,6.847800e+11,6.397040e+11,6.099140e+11,5.961046e+11,6.001064e+11,6.058029e+11,6.487983e+11,3.138310e+08,3.159937e+08,3.183010e+08,3.206352e+08,3.229413e+08,3.249855e+08,3.266875e+08,4.385210e+12,0.514928


# **Step 2: Data Visualization**

*2.1 Make a new GDP dataset*

*   Reshape the dataset 
* row become the column







In [8]:
GDP = GDP.rename(columns={'Country Name':'Country_Name','GDP_2012':'2012',
                           'GDP_2013':'2013','GDP_2014':'2014','GDP_2015': '2015','GDP_2016':'2016',
                           'GDP_2017':'2017','GDP_2018':'2018'})
GDP = GDP.drop(columns = ['Country Code'])

df1 = GDP.set_index('Country_Name')   #new GDP form
df1 = df1.stack()
df1.index = df1.index.rename('Year',level = 1)
df1.name = 'GDP'
df1 = df1.reset_index()
df1

,Country_Name,Year,GDP
0,China,2012,8.532231e+12
1,China,2013,9.570406e+12
2,China,2014,1.047568e+13
3,China,2015,1.106155e+13
4,China,2016,1.123328e+13
...,...,...,...
65,United States,2014,1.752175e+13
66,United States,2015,1.821930e+13
67,United States,2016,1.870719e+13
68,United States,2017,1.948539e+13


*2.2 Make a new Military dataset*

*   Reshape the dataset 
* row become the column




In [9]:
# convert to new Military form

Mili = Mili.rename(columns={'Country Name':'Country_Name','Mili_2012':'2012',
                           'Mili_2013':'2013','Mili_2014':'2014','Mili_2015': '2015','Mili_2016':'2016',
                           'Mili_2017':'2017','Mili_2018':'2018'})
Mili = Mili.drop(columns = ['Country Code'])
Mili

df2 = Mili.set_index('Country_Name') #new Military dataset
df2 = df2.stack()
df2.index = df2.index.rename('Year',level = 1)
df2.name = 'Military_spend'
df2 = df2.reset_index()
df2

,Country_Name,Year,Military_spend
0,China,2012,1.573904e+11
1,China,2013,1.798805e+11
2,China,2014,2.007722e+11
3,China,2015,2.140931e+11
4,China,2016,2.160313e+11
...,...,...,...
65,United States,2014,6.099140e+11
66,United States,2015,5.961046e+11
67,United States,2016,6.001064e+11
68,United States,2017,6.058029e+11


*2.3 Make a New Population dataset*
*   Reshape the dataset 
* row become the column


In [10]:
Pop = Pop.rename(columns={'Country Name':'Country_Name','Pop_2012':'2012',
                           'Pop_2013':'2013','Pop_2014':'2014','Pop_2015': '2015','Pop_2016':'2016',
                           'Pop_2017':'2017','Pop_2018':'2018'})
Pop = Pop.drop(columns = ['Country Code'])

df3 = Pop.set_index('Country_Name') #new Pop dataset
df3 = df3.stack()
df3.index = df3.index.rename('Year',level = 1)
df3.name = 'Population'
df3 = df3.reset_index()
df3


,Country_Name,Year,Population
0,China,2012,1.350695e+09
1,China,2013,1.357380e+09
2,China,2014,1.364270e+09
3,China,2015,1.371220e+09
4,China,2016,1.378665e+09
...,...,...,...
65,United States,2014,3.183010e+08
66,United States,2015,3.206352e+08
67,United States,2016,3.229413e+08
68,United States,2017,3.249855e+08


**2.4 Military spending VS  GDP spending**

* 2.4.1. Merge GDP + military spending dataset 

In [11]:
# Merge to a new dataset 
gdp_Mili = pd.merge(df1,df2,on=['Country_Name','Year'])
gdp_Mili = pd.merge(gdp_Mili,df3,on= ['Country_Name','Year'])
gdp_Mili['GDP_per_person'] = gdp_Mili['GDP'] / gdp_Mili['Population']            #find the gdp per capita
gdp_Mili['Mili_per_person'] = gdp_Mili['Military_spend']/ gdp_Mili['Population']    #find the military per capita

gdp_Mili







,Country_Name,Year,GDP,Military_spend,Population,GDP_per_person,Mili_per_person
0,China,2012,8.532231e+12,1.573904e+11,1.350695e+09,6316.918863,116.525476
1,China,2013,9.570406e+12,1.798805e+11,1.357380e+09,7050.645920,132.520334
2,China,2014,1.047568e+13,2.007722e+11,1.364270e+09,7678.599432,147.164567
3,China,2015,1.106155e+13,2.140931e+11,1.371220e+09,8066.942424,156.133275
4,China,2016,1.123328e+13,2.160313e+11,1.378665e+09,8147.938148,156.695992
...,...,...,...,...,...,...,...
65,United States,2014,1.752175e+13,6.099140e+11,3.183010e+08,55047.725560,1916.154786
66,United States,2015,1.821930e+13,5.961046e+11,3.206352e+08,56822.518820,1859.136825
67,United States,2016,1.870719e+13,6.001064e+11,3.229413e+08,57927.516852,1858.252328
68,United States,2017,1.948539e+13,6.058029e+11,3.249855e+08,59957.725851,1864.091950


* 2.4.2 Select each country's own dataset

In [12]:
dfChina = gdp_Mili.loc[0:6]
dfGM = gdp_Mili.loc[7:13]
dfFrance = gdp_Mili.loc[14:20]
dfUK = gdp_Mili.loc[21:27]
dfIs = gdp_Mili.loc[28:34]
dfItaly = gdp_Mili.loc[35:41]
dfKorean = gdp_Mili.loc[42:48]
dfRussian = gdp_Mili.loc[49:55]
dfSA = gdp_Mili.loc[56:62]
dfUS = gdp_Mili.loc[63:69]

In [13]:
#Chenck
dfChina

,Country_Name,Year,GDP,Military_spend,Population,GDP_per_person,Mili_per_person
0,China,2012,8.532231e+12,1.573904e+11,1.350695e+09,6316.918863,116.525476
1,China,2013,9.570406e+12,1.798805e+11,1.357380e+09,7050.645920,132.520334
2,China,2014,1.047568e+13,2.007722e+11,1.364270e+09,7678.599432,147.164567
3,China,2015,1.106155e+13,2.140931e+11,1.371220e+09,8066.942424,156.133275
4,China,2016,1.123328e+13,2.160313e+11,1.378665e+09,8147.938148,156.695992
5,China,2017,1.231041e+13,2.278294e+11,1.386395e+09,8879.438149,164.332256
6,China,2018,1.389482e+13,2.499969e+11,1.392730e+09,9976.676822,179.501340


* 2.4.3 **Graph** military spending VS GDP spending  (By country)
 * China, German, UK, France, Israel, Italy, Korean, Russian, Saudi Arabia, US


In [14]:
#Grah GDP spend in absolute value by line graph 

trace1 = go.Scatter(
                    y = dfChina['GDP'],
                    x = dfChina.Year,
                    mode = "lines+markers",
                    name = "GDP of China",
                    marker = dict(color = '#d4546e'),
                    text= "China ")

trace2 = go.Scatter(
                    y = dfFrance['GDP'],
                    x = dfFrance.Year,
                    mode = "lines+markers",
                    name = "GDP of France",
                    marker = dict(color = '#e56f63'),
                    text= "France ")
trace3 = go.Scatter(
                    y = dfGM['GDP'],
                    x = dfGM.Year,
                    mode = "lines+markers",
                    name = "GDP of German",
                    marker = dict(color = '#e48190'),
                    text= "German")
trace4 = go.Scatter(
                    y = dfIs['GDP'],
                    x = dfIs.Year,
                    mode = "lines+markers",
                    name = "GDP of Israel",
                    marker = dict(color = '#77b594'),
                    text= "Israel")
trace5 = go.Scatter(
                    y = dfItaly['GDP'],
                    x = dfItaly.Year,
                    mode = "lines+markers",
                    name = "GDP of Italy",
                    marker = dict(color = '#d1cf84'),
                    text= "Italy")
trace6 = go.Scatter(
                    y = dfKorean['GDP'],
                    x = dfKorean.Year,
                    mode = "lines+markers",
                    name = "GDP of Korean",
                    marker = dict(color = '#cfe7ac'),
                    text= "Korean")
trace7 = go.Scatter(
                    y = dfRussian['GDP'],
                    x = dfRussian.Year,
                    mode = "lines+markers",
                    name = "GDP of Russian",
                    marker = dict(color = '#ffffbd'),
                    text= "Russian")
trace8 = go.Scatter(
                    y = dfSA['GDP'],
                    x = dfSA.Year,
                    mode = "lines+markers",
                    name = "GDP of Saudi Arabia",
                    marker = dict(color = 'hsl(120,100%,20%)'),
                    text= "Saudi Arabia")

trace9 = go.Scatter(
                    y = dfUK['GDP'],
                    x = dfUK.Year,
                    mode = "lines+markers",
                    name = "GDP of UK",
                    marker = dict(color = '#f9d5d9'),
                    text= "UK")
trace10 = go.Scatter(
                    y = dfUS['GDP'],
                    x = dfUS.Year,
                    mode = "lines+markers",
                    name = "GDP of US",
                    marker = dict(color = '#c2104e'),
                    text= "US")

line = [trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10]

fig = go.Figure(line)
fig.update_layout(
     title={
        'text': 'The GDP among 10 countries (2012 - 2018)',
        'y':0.96,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Year',
    yaxis_title = 'GDP spend (current USD$)'
)

fig.show()


In [15]:
# line Graph of military spending for 10 countries (absulote)

trace1 = go.Scatter(
                    y = dfChina['Military_spend'],
                    x = dfChina.Year,
                    mode = "lines+markers",
                    name = "China Military spending",
                    marker = dict(color = '#f2d675'),
                    text= "China ")

trace2 = go.Scatter(
                    y = dfFrance['Military_spend'],
                    x = dfFrance.Year,
                    mode = "lines+markers",
                    name = "France Military spending",
                    marker = dict(color = '#fff34d'),
                    text= "France ")
trace3 = go.Scatter(
                    y = dfGM['Military_spend'],
                    x = dfGM.Year,
                    mode = "lines+markers",
                    name = "German Military spending",
                    marker = dict(color = '#66a598'),
                    text= "German")
trace4 = go.Scatter(
                    y = dfIs['Military_spend'],
                    x = dfIs.Year,
                    mode = "lines+markers",
                    name = "Israel Military spending",
                    marker = dict(color = '#77b594'),
                    text= "Israel")
trace5 = go.Scatter(
                    y = dfItaly['Military_spend'],
                    x = dfItaly.Year,
                    mode = "lines+markers",
                    name = "Italy Military spending",
                    marker = dict(color = '#358f8a'),
                    text= "Italy")
trace6 = go.Scatter(
                    y = dfKorean['Military_spend'],
                    x = dfKorean.Year,
                    mode = "lines+markers",
                    name = "Korean Military spending",
                    marker = dict(color = '#b3d0c2'),
                    text= "Korean")
trace7 = go.Scatter(
                    y = dfRussian['Military_spend'],
                    x = dfRussian.Year,
                    mode = "lines+markers",
                    name = "Russian Military spending",
                    marker = dict(color = '#d9e6da'),
                    text= "Russian")
trace8 = go.Scatter(
                    y = dfSA['Military_spend'],
                    x = dfSA.Year,
                    mode = "lines+markers",
                    name = "Saudi Arabia Military spending",
                    marker = dict(color = '#e7efba'),
                    text= "Saudi Arabia")

trace9 = go.Scatter(
                    y = dfUK['Military_spend'],
                    x = dfUK.Year,
                    mode = "lines+markers",
                    name = "UK Military spending",
                    marker = dict(color = '#8fbaaa'),
                    text= "UK")
trace10 = go.Scatter(
                    y = dfUS['Military_spend'],
                    x = dfUS.Year,
                    mode = "lines+markers",
                    name = "Russian Military spending",
                    marker = dict(color = '#ffc654'),
                    text= "US")

line = [trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10]

fig = go.Figure(line)
fig.update_layout(
     title={
        'text': 'The military spending among 10 countries (2012 - 2018)',
        'y':0.96,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Year',
    yaxis_title = 'Military spend (current USD$)'
)

fig.show()


In [16]:
#Graph the population among 10 countries

trace1 = go.Scatter(
                    y = dfChina['Population'],
                    x = dfChina.Year,
                    mode = "lines+markers",
                    name = "China Population",
                    marker = dict(color = '#8c0b0b'),
                    text= "China ")

trace2 = go.Scatter(
                    y = dfFrance['Population'],
                    x = dfFrance.Year,
                    mode = "lines+markers",
                    name = "France Population",
                    marker = dict(color = '#e4c5be'),
                    text= "France ")
trace3 = go.Scatter(
                    y = dfGM['Population'],
                    x = dfGM.Year,
                    mode = "lines+markers",
                    name = "German Population",
                    marker = dict(color = '#d39a8d'),
                    text= "German")
trace4 = go.Scatter(
                    y = dfIs['Population'],
                    x = dfIs.Year,
                    mode = "lines+markers",
                    name = "Israel Population",
                    marker = dict(color = '#de425b'),
                    text= "Israel")
trace5 = go.Scatter(
                    y = dfItaly['Population'],
                    x = dfItaly.Year,
                    mode = "lines+markers",
                    name = "Italy Population",
                    marker = dict(color = '#eeadad'),
                    text= "Italy")
trace6 = go.Scatter(
                    y = dfKorean['Population'],
                    x = dfKorean.Year,
                    mode = "lines+markers",
                    name = "Korean Population",
                    marker = dict(color = '#e88b8d'),
                    text= "Korean")
trace7 = go.Scatter(
                    y = dfRussian['Population'],
                    x = dfRussian.Year,
                    mode = "lines+markers",
                    name = "Russian Population",
                    marker = dict(color = '#d39a8d'),
                    text= "Russian")
trace8 = go.Scatter(
                    y = dfSA['Population'],
                    x = dfSA.Year,
                    mode = "lines+markers",
                    name = "Saudi Arabia Population",
                    marker = dict(color = '#df676e'),
                    text= "Saudi Arabia")

trace9 = go.Scatter(
                    y = dfUK['Population'],
                    x = dfUK.Year,
                    mode = "lines+markers",
                    name = "UK Population",
                    marker = dict(color = '#f1cfce'),
                    text= "UK")
trace10 = go.Scatter(
                    y = dfUS['Population'],
                    x = dfUS.Year,
                    mode = "lines+markers",
                    name = "Russian Population",
                    marker = dict(color = '#a74434'),
                    text= "US")

line = [trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10]

fig = go.Figure(line)
fig.update_layout(
     title={
        'text': 'The Population among 10 countries (2012 - 2018)',
        'y':0.96,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Year',
    yaxis_title = 'Population （Billion）'
)

fig.show()



**2.5 Compare Overall military spending in 10 countries**

* **2.5.1 In absolute  value**

In [17]:
# line graph:  present the overall military spending
trace1 = go.Scatter(
                    y = data.Mili_2012,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2012 Military spending",
                    marker = dict(color = '#de425b'),
                    text= "2012 ")

trace2 = go.Scatter(
                    y = data.Mili_2013,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2013 Military spending",
                    marker = dict(color = '#e56f63'),
                    text= "2013 ")
trace3 = go.Scatter(
                    y = data.Mili_2014,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2014 Military spending",
                    marker = dict(color = '#efbe8a'),
                    text= "2014")
trace4 = go.Scatter(
                    y = data.Mili_2015,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2015 Military spending",
                    marker = dict(color = '#77b594'),
                    text= "2015")
trace5 = go.Scatter(
                    y = data.Mili_2016,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2016 Military spending",
                    marker = dict(color = '#a1ce9f'),
                    text= "2016 ")
trace6 = go.Scatter(
                    y = data.Mili_2017,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2017 Military spending",
                    marker = dict(color = '#cfe7ac'),
                    text= "2017")
trace7 = go.Scatter(
                    y = data.Mili_2018,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "2018 Military spending",
                    marker = dict(color = '#ffffbd'),
                    text= "2018")
trace8 = go.Scatter(
                    y = data.Overall_Miliraty_spending,
                    x = data.Country_Name,
                    mode = "lines+markers",
                    name = "Overall Military spending",
                    marker = dict(color = '#cc6074'),
                    text= "2012-2018 total spending")

line = [trace1, trace2,trace3,trace4,trace5,trace6,trace7,trace8]

fig = go.Figure(line)
fig.update_layout(
     title={
        'text': 'The overall military spending among 10 countries (2012 - 2018)',
        'y':0.96,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Country',
    yaxis_title = 'Military spend in Trillion ($USD)'
)

fig.show()


* *From 2012 to 2018, The total cost of the US military is the highest among the ten countries*

* **2.5.2 In percentage**


In [18]:
# Use pie chart show the overall military spending in percentage
import plotly.express as px
fig = px.pie(data_frame = data, values = data.Overall_Miliraty_spending,names=data.Country_Name, 
             title='Overal military spending in percentage for 10 countries',color_discrete_sequence=px.colors.sequential.RdBu,
             hover_name = data.Country_Name)
fig.show()


* *The United States has the largest percentage of the 10 countries on military expenditures, around 51.5%*

**2.6 Compare the military spending per capital with the GDP per capital**

  * Military expenditure per capita  VS   GDP per capita


* **2.6.1 In absolute**

In [19]:
#line graph for GDP per capita (in absolute value)

trace1 = go.Scatter(
                    y = dfChina['GDP_per_person'],
                    x = dfChina.Year,
                    mode = "lines+markers",
                    name = "China",
                    marker = dict(color = '#cc6074'),
                    text= "China ")

trace2 = go.Scatter(
                    y = dfFrance['GDP_per_person'],
                    x = dfFrance.Year,
                    mode = "lines+markers",
                    name = "France ",
                    marker = dict(color = '#e5e4d0'),
                    text= "France ")
trace3 = go.Scatter(
                    y = dfGM['GDP_per_person'],
                    x = dfGM.Year,
                    mode = "lines+markers",
                    name = "German",
                    marker = dict(color = '#9abb94'),
                    text= "German")
trace4 = go.Scatter(
                    y = dfIs['GDP_per_person'],
                    x = dfIs.Year,
                    mode = "lines+markers",
                    name = "Israel",
                    marker = dict(color = '#77b594'),
                    text= "Israel")
trace5 = go.Scatter(
                    y = dfItaly['GDP_per_person'],
                    x = dfItaly.Year,
                    mode = "lines+markers",
                    name = "Italy",
                    marker = dict(color = '#f1e0ce'),
                    text= "Italy")
trace6 = go.Scatter(
                    y = dfKorean['GDP_per_person'],
                    x = dfKorean.Year,
                    mode = "lines+markers",
                    name = "Korean",
                    marker = dict(color = '#e9c2aa'),
                    text= "Korean")
trace7 = go.Scatter(
                    y = dfRussian['GDP_per_person'],
                    x = dfRussian.Year,
                    mode = "lines+markers",
                    name = "Russian",
                    marker = dict(color = '#d9827c'),
                    text= "Russian")
trace8 = go.Scatter(
                    y = dfSA['GDP_per_person'],
                    x = dfSA.Year,
                    mode = "lines+markers",
                    name = "Saudi Arabia",
                    marker = dict(color = '#e1a38e'),
                    text= "Saudi Arabia")

trace9 = go.Scatter(
                    y = dfUK['GDP_per_person'],
                    x = dfUK.Year,
                    mode = "lines+markers",
                    name = "UK",
                    marker = dict(color = '#69a882'),
                    text= "UK")
trace10 = go.Scatter(
                    y = dfUS['GDP_per_person'],
                    x = dfUS.Year,
                    mode = "lines+markers",
                    name = "US",
                    marker = dict(color = '#1e9478'),
                    text= "US")

line = [trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10]

fig = go.Figure(line)
fig.update_layout(
     title={
        'text': 'The GDP per capita among 10 countries (2012 - 2018)',
        'y':0.96,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Year',
    yaxis_title = 'GDP per capita (current USD$)'
)

fig.show()


In [20]:
#line graph for military spending per capita ( in absolute value)


trace1 = go.Scatter(
                    y = dfChina['Mili_per_person'],
                    x = dfChina.Year,
                    mode = "lines+markers",
                    name = "China",
                    marker = dict(color = '#cc6074'),
                    text= "China ")

trace2 = go.Scatter(
                    y = dfFrance['Mili_per_person'],
                    x = dfFrance.Year,
                    mode = "lines+markers",
                    name = "France",
                    marker = dict(color = '#c1bad1'),
                    text= "France ")
trace3 = go.Scatter(
                    y = dfGM['Mili_per_person'],
                    x = dfGM.Year,
                    mode = "lines+markers",
                    name = "German",
                    marker = dict(color = '#e9bed5'),
                    text= "German")
trace4 = go.Scatter(
                    y = dfIs['Mili_per_person'],
                    x = dfIs.Year,
                    mode = "lines+markers",
                    name = "Israel",
                    marker = dict(color = '#9c9cbd'),
                    text= "Israel")
trace5 = go.Scatter(
                    y = dfItaly['Mili_per_person'],
                    x = dfItaly.Year,
                    mode = "lines+markers",
                    name = "Italy",
                    marker = dict(color = '#e19fb8'),
                    text= "Italy")
trace6 = go.Scatter(
                    y = dfKorean['Mili_per_person'],
                    x = dfKorean.Year,
                    mode = "lines+markers",
                    name = "Korean",
                    marker = dict(color = '#f2ddec'),
                    text= "Korean")
trace7 = go.Scatter(
                    y = dfRussian['Mili_per_person'],
                    x = dfRussian.Year,
                    mode = "lines+markers",
                    name = "Russian",
                    marker = dict(color = '#e19fb8'),
                    text= "Russian")
trace8 = go.Scatter(
                    y = dfSA['Mili_per_person'],
                    x = dfSA.Year,
                    mode = "lines+markers",
                    name = "Saudi Arabia",
                    marker = dict(color = '#416894'),
                    text= "Saudi Arabia")

trace9 = go.Scatter(
                    y = dfUK['Mili_per_person'],
                    x = dfUK.Year,
                    mode = "lines+markers",
                    name = "UK",
                    marker = dict(color = '#e2d9e7'),
                    text= "UK")
trace10 = go.Scatter(
                    y = dfUS['Mili_per_person'],
                    x = dfUS.Year,
                    mode = "lines+markers",
                    name = "US",
                    marker = dict(color = '#7281a9'),
                    text= "US")

line = [trace1,trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10]

fig = go.Figure(line)
fig.update_layout(
     title={
        'text': 'Military expenditure per capita among 10 countries (2012 - 2018)',
        'y':0.96,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Year',
    yaxis_title = 'Military expenditure per capita (current USD$)'
)

fig.show()


  * Before compre in percentage:
    * Reshape the GDP/ perosn dataset make it easier for using pie chart.

In [21]:
#GDP/Person dataframe  (row to coloumn)

newdf = pd.pivot(gdp_Mili, index="Country_Name", columns="Year", values="GDP_per_person")
newdf = newdf.rename_axis(columns=None).reset_index()
newdf

,Country_Name,2012,2013,2014,2015,2016,2017,2018
0,China,6316.918863,7050.645920,7678.599432,8066.942424,8147.938148,8879.438149,9976.676822
1,France,40874.703505,42592.934094,43011.263103,36638.184929,37037.374186,38812.161034,41631.090739
2,Germany,43858.363055,46285.764069,47959.993274,41139.544569,42098.920445,44349.589655,47639.003441
3,Israel,32511.237852,36309.468826,37678.889774,35776.795171,37321.624037,40541.862091,41719.725442
4,Italy,35053.526244,35549.974697,35518.415292,30230.226302,30939.714246,32406.720315,34520.085162
5,"Korea, Rep.",25466.760517,27182.734310,29249.575221,28732.231076,29288.870439,31616.843400,33340.265152
6,Russian Federation,15420.874541,15974.644627,14318.222415,9462.253160,8845.543753,10894.359188,11555.978814
7,Saudi Arabia,25243.358698,24844.744371,24463.903164,20627.932782,19879.297530,20803.742566,23338.963458
8,United Kingdom,42462.771568,43444.533002,47425.607681,44974.831877,41064.133432,40361.417383,43043.227816
9,United States,51610.605278,53117.667831,55047.725560,56822.518820,57927.516852,59957.725851,62840.020239


 2. Reshape Military / person datasete make it easier for using pie chart.

In [22]:
# Mili / person datafram
newdf1 = pd.pivot(gdp_Mili, index="Country_Name", columns="Year", values="Mili_per_person")
newdf1 = newdf1.rename_axis(columns=None).reset_index()
newdf1

,Country_Name,2012,2013,2014,2015,2016,2017,2018
0,China,116.525476,132.520334,147.164567,156.133275,156.695992,164.332256,179.501340
1,France,914.336406,945.732438,959.306081,831.608844,859.635589,903.582741,952.718700
2,Germany,577.810325,569.535569,569.291798,487.381907,504.920060,549.036633,596.709019
3,Israel,1967.902896,2148.980455,2250.061420,2024.967731,1729.910455,1788.255704,1795.243459
4,Italy,500.187265,497.351525,455.690512,365.233534,412.898911,436.890167,460.223501
5,"Korea, Rep.",636.491123,680.388140,739.996079,716.863811,720.165280,762.640668,834.582123
6,Russian Federation,568.913553,615.669616,588.907672,460.930957,479.729528,460.406954,424.892416
7,Saudi Arabia,1937.832423,2230.095994,2612.233259,2748.810834,1962.631076,2126.943030,2004.592668
8,United Kingdom,918.296064,886.687836,916.110733,827.170040,733.390874,702.908044,752.286093
9,United States,2182.002485,2024.420011,1916.154786,1859.136825,1858.252328,1864.091950,1985.990499


* **2.6.2 In percetage**
    * Compare in each year as the population might change for each year

In [23]:
from plotly.subplots import make_subplots
labels = newdf.Country_Name
#labels = ['China','German','UK','France','Israel','Italy','Korean','Russian','Saudi Arabia','US']
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])


#2012
values_gdp = newdf['2012'] #GDP/person
values_mili = newdf1['2012']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


                      
# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


fig4.update_layout(title_text = ' 2012 GDP per capita VS Military expenditure per capita',
                  annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='Military expenditure per capita', x=0.90, y=0.5, font_size=12, showarrow=False)])


fig4.show()



* 2013

In [24]:
labels = newdf.Country_Name
#labels = ['China','German','UK','France','Israel','Italy','Korean','Russian','Saudi Arabia','US']
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])



#2013
values_gdp = newdf['2013'] #GDP/person
values_mili = newdf1['2013']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


#fig4.update_traces(hoverinfo = 'label + pervent+name')
fig4.update_layout(title_text = ' 2013 GDP per capita VS military expenditure per capita',
                  annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='military expenditure per capita', x=0.90, y=0.5, font_size=12, showarrow=False)])
fig4.show()


* 2014

In [25]:
labels = newdf.Country_Name
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])



#2014
values_gdp = newdf['2014'] #GDP/person
values_mili = newdf1['2014']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


#fig4.update_traces(hoverinfo = 'label + pervent+name')
fig4.update_layout(title_text = ' 2014 GDP per capita VS Military expenditure per capita by percentage',
                   annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='Military expenditure per capita', x=0.85, y=0.5, font_size=12, showarrow=False)])
fig4.show()

* 2015 

In [26]:
labels = newdf.Country_Name
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])



#2015
values_gdp = newdf['2015'] #GDP/person
values_mili = newdf1['2015']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


#fig4.update_traces(hoverinfo = 'label + pervent+name')
fig4.update_layout(title_text = ' 2015 GDP per capita VS Military expenditure per capita',
                   annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='Military expenditure per capita', x=0.85, y=0.5, font_size=12, showarrow=False)])
fig4.show()

* 2016

In [27]:
labels = newdf.Country_Name
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])



#2016
values_gdp = newdf['2016'] #GDP/person
values_mili = newdf1['2016']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


#fig4.update_traces(hoverinfo = 'label + pervent+name')
fig4.update_layout(title_text = ' 2016 GDP per capita VS Military expenditure per capita',
                   annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='military expenditure per capita', x=0.85, y=0.5, font_size=12, showarrow=False)])
fig4.show()

* 2017

In [28]:
labels = newdf.Country_Name
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])



#2017
values_gdp = newdf['2017'] #GDP/person
values_mili = newdf1['2017']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


#fig4.update_traces(hoverinfo = 'label + pervent+name')
fig4.update_layout(title_text = ' 2017 GDP per capita VS Military expenditure per capita',
                   annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='Military expenditure per capita', x=0.85, y=0.5, font_size=12, showarrow=False)])
fig4.show()

* 2018

In [29]:
labels = newdf.Country_Name
#make color
sunflowers_colors = ['#de425b', '#a1acc5', '#526c9a',
                     '#f3d0d1', '#c9cedb','#f2afb2','#e76b77',
                     '#ee8e94','#7a8baf','#204f85']
fig4 = make_subplots(rows = 1, cols = 2, specs= [[{'type': 'domain'},{'type' : 'domain'}]])



#2018
values_gdp = newdf['2018'] #GDP/person
values_mili = newdf1['2018']  #mili / person                         

#GDP/person
fig4.add_trace(go.Pie(labels = labels, values= values_gdp,
               scalegroup='one', marker_colors=sunflowers_colors),
                         1, 1)
# Military/person
fig4.add_trace(go.Pie(labels = labels, values= values_mili),
                         1, 2)


# Use `hole` to create a donut-like pie chart
fig4.update_traces(hole=.5, hoverinfo="label+percent+name")                     


#fig4.update_traces(hoverinfo = 'label + pervent+name')
fig4.update_layout(title_text = ' 2018 GDP per capital VS Military expenditure per capita',
                   annotations=[dict(text='GDP per capita', x=0.14, y=0.5, font_size=14, showarrow=False),
                 dict(text='Military expenditure per capita', x=0.85, y=0.5, font_size=12, showarrow=False)])
fig4.show()

**2.7 Single out the fastest growing countries in military spending**

 * **2.7.1 In absolute value**


In [30]:
#Calculate the percentage and difference for each
def growing_fast(DF):  
  DF['mili_grow'] = DF.Military_spend.pct_change().mul(100).round(3).map(lambda x: '{0:g}%'.format(x) if x==x else x) #find the growth rate
  #DF.drop(DF.tail(1).index,inplace=True)
  DF['mili_difference'] = DF.Military_spend.diff()
  return DF

growing_fast(dfChina)
growing_fast(dfFrance)
growing_fast(dfGM)
growing_fast(dfIs)
growing_fast(dfItaly)
growing_fast(dfKorean)
growing_fast(dfRussian)
growing_fast(dfSA)
growing_fast(dfUK)
growing_fast(dfUS)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country_Name,Year,GDP,Military_spend,Population,GDP_per_person,Mili_per_person,mili_grow,mili_difference
63,United States,2012,1.619701e+13,6.847800e+11,313830990.0,51610.605278,2182.002485,NaN,NaN
64,United States,2013,1.678485e+13,6.397040e+11,315993715.0,53117.667831,2024.420011,-6.583%,-4.507600e+10
65,United States,2014,1.752175e+13,6.099140e+11,318301008.0,55047.725560,1916.154786,-4.657%,-2.979000e+10
66,United States,2015,1.821930e+13,5.961046e+11,320635163.0,56822.518820,1859.136825,-2.264%,-1.380936e+10
67,United States,2016,1.870719e+13,6.001064e+11,322941311.0,57927.516852,1858.252328,0.671%,4.001804e+09
68,United States,2017,1.948539e+13,6.058029e+11,324985539.0,59957.725851,1864.091950,0.949%,5.696484e+09
69,United States,2018,2.052905e+13,6.487983e+11,326687501.0,62840.020239,1985.990499,7.097%,4.299535e+10


In [31]:
#Graph
# Using the Bar chart to show the increase military spending in absolute vaue

fig = go.Figure()

#China
fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfChina.mili_difference,
    name='China',
    marker_color='#de141b',
    text = 'China'
))

#France
fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfFrance.mili_difference,
    name='French',
    marker_color='#f3806c',
    text = 'France'
))

# German

fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfFrance.mili_difference,
    name='German',
    marker_color='#edad82',
    text = 'German'
))

#Israel
fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfIs.mili_difference,
    name='Israel',
    marker_color='#f0d0b5',
    text = 'Israel'
))



#Italy

fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfItaly.mili_difference,
    name='Italy',
    marker_color='#d3c49e',
    text = 'Italy'
))

#korean


fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfKorean.mili_difference,
    name='Korean',
    marker_color='lightsalmon',
    text = 'Korean'
))


#Russian

fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfRussian.mili_difference,
    name='Russian',
    marker_color='#bab178',
    text = 'Russian'
))


#Saudi Arabia

fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfSA.mili_difference,
    name='Saudi Arabia',
    marker_color='#7a8f33',
    text = 'Saudi Arabia'
))


#UK

fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfUK.mili_difference,
    name='UK',
    marker_color='#de6884',
    text = 'UK'
))



#US

fig.add_trace(go.Bar(
    x=gdp_Mili.Year,
    y=dfUS.mili_difference,
    name='US',
    marker_color='#9b5859',
    text = 'US'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title={
        'text': 'The increase of national military expenditure',
        'y':0.96,
        'x':0.37,
        'xanchor': 'center',
        'yanchor': 'top'},
                  xaxis_title = 'Year',
                  yaxis_title = 'Military spending difference (Billions dollars $USD)')
fig.show()


* **2.7.2 For percentage value**

In [32]:
# Using the line graph  to show the growth rate %.
trace1 = go.Scatter(
                    y = dfChina.mili_grow,
                    x = dfChina.Year,
                    mode = "lines+markers",
                    name = "Growth rate of China military expenditure%",
                    marker = dict(color = '#de425b'),
                    text= "China ")

trace2 = go.Scatter(
                    x = dfFrance.Year,
                    y = dfFrance.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of French military expenditure %",
                    marker = dict(color = '#e56f63'),
                    text= "France")
trace3 = go.Scatter(
                    x = dfGM.Year,
                    y = dfGM.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of German military expenditure %",
                    marker = dict(color = '#efbe8a'),
                    text= "German")
trace4 = go.Scatter(
                    x = dfIs.Year,
                    y = dfIs.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of Israel military expenditure %",
                    marker = dict(color = '#77b594'),
                    text= "Israel")
trace5 = go.Scatter(
                    x = dfItaly.Year,
                    y = dfItaly.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of Italy military expenditure %",
                    marker = dict(color = '#ffe6c3'),
                    text= "Italy")
trace6 = go.Scatter(
                    x = dfKorean.Year,
                    y = dfKorean.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of Korean military expenditure %",
                    marker = dict(color = '#5090aa'),
                    text= "Korean")
trace7 = go.Scatter(
                    x = dfRussian.Year,
                    y = dfRussian.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of Russian military expenditure %",
                    marker = dict(color = '#ffffbd'),
                    text= "Russian")
trace8 = go.Scatter(
                    x = dfSA.Year,
                    y = dfSA.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of Saudi Arabia military expenditure%",
                    marker = dict(color = '#e5c6b7'),
                    text= "Saudi Arabia")

trace9 = go.Scatter(
                    x = dfUK.Year,
                    y = dfUK.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of UK military expenditure%",
                    marker = dict(color = '#e6c0cf'),
                    text= "UK")

trace10 = go.Scatter(
                    x = dfUS.Year,
                    y = dfUS.mili_grow,
                    mode = "lines+markers",
                    name = "Growth rate of US military expenditure%",
                    marker = dict(color = '#9a536d'),
                    text= "US")

line = [trace1, trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10]

fig = go.Figure(line)
fig.update_layout(
    title={
        'text': 'Military expenditure growth rate in percentage (%)',
        'y':0.96,
        'x':0.37,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = 'Year',
    yaxis_title = 'Growth rate of military spednding %'
)

fig.show()


Looks like South Korean is the fastest growing countries in military spending in percentage